# Here we have rewritten PREOB() function.

$\textbf{Short algorithm:}$ open file $\to$ use formula $\to$ calculate mean $\to$ save file 

Additionaly, we plotted , пожалуйста,graphics. 

# Libraries

In [ ]:
import numpy as np
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt
import cv2 #!pip3 install opencv-python
from scipy import signal
from scipy.signal import find_peaks

# Open txt files

Open file $\to$ make a list with data $\to$ make an array from list

In [ ]:
# file_call1 = '/home/adel/Dropbox/QC2Python(spectrums)/Files/call1.txt'
file_call1 = 'Files/call1.txt'

data_call1 = [] #to work with data from txt file we should write it in array, so here we create an empty list
with open(file_call1) as file:
    next(file)
    for line in file:
        row = line.split()
        row = [int(x) for x in row]
        data_call1.append(row) #add data from txt the list

In [ ]:
# file_call2 = '/home/adel/Dropbox/QC2Python(spectrums)/Files/call2.txt'
file_call2 = 'Files/call2.txt'

data_call2 = [] #to work with data from txt file we should write it in array, so here we create an empty list 
with open(file_call2) as file:
    next(file)
    for line in file:
        row = line.split()
        row = [int(x) for x in row]
        data_call2.append(row) #add data from txt to the list

In [ ]:
# file_call3 = '/home/adel/Dropbox/QC2Python(spectrums)/Files/call3.txt'
file_call3 = 'Files/call3.txt'


data_call3 = [] #to work with data from txt file we should write it in array, so here we create an empty list
with open(file_call3) as file:
    next(file)
    for line in file:
        row = line.split()
        row = [int(x) for x in row]
        data_call3.append(row) #add data from txt to the list

In [ ]:
# file_call4 = '/home/adel/Dropbox/QC2Python(spectrums)/Files/call4.txt'
file_call4 = 'Files/call4.txt'

data_call4 = [] #to work with data from txt file we should write it in array, so here we create an empty list 
with open(file_call4) as file:
    next(file)
    for line in file:
        row = line.split()
        row = [int(x) for x in row]
        data_call4.append(row) #add data from txt to the list

In [ ]:
o_call1 = np.array(data_call1) #make an array from list
o_call2 = np.array(data_call2) #make an array from list
o_call3 = np.array(data_call3) #make an array from list
o_call4 = np.array(data_call4) #make an array from list

### Save array as image

In [ ]:
plt.imsave('pic_1', o_call1) #сохраняем массив из текста в картинку (чтобы на всякие случай проверить поворот картинки)
plt.imsave('pic_2', o_call2) #сохраняем массив из текста в картинку (чтобы на всякие случай проверить поворот картинки)
plt.imsave('pic_3', o_call3) #сохраняем массив из текста в картинку (чтобы на всякие случай проверить поворот картинки)
plt.imsave('pic_4', o_call4) #сохраняем массив из текста в картинку (чтобы на всякие случай проверить поворот картинки)

# Rotate and crop image

In [ ]:
def subimage(name_file, center, theta, width, height):
    ''' 
    Rotates OpenCV image around center with angle theta (in deg)
    then crops the image according to width and height.
    '''
    path = os.getcwd()#для просмотра корневой дирректории
    image = cv2.imread(f'{path}/{name_file}') #открываем картинку (не повёрнута, не обрезана)
    
    if center[0]<1 and center[1]<1 and width<1: #проверка на случай если мы работаем в относительных координатах и их пересчёт к абсолютным
        center_x = center[0]*image.shape[1]
        center_y = center[1]*image.shape[0]
        width = width * image.shape[1]
        height = height * image.shape[0]
        center = (center_x,center_y)
    shape = ( image.shape[1], image.shape[0] ) # cv2.warpAffine expects shape in (length, height)
    matrix = cv2.getRotationMatrix2D( center=center, angle=theta, scale=1 ) #поворот матрицы вокруг указанного центра
    image = cv2.warpAffine(src=image, M=matrix, dsize=shape ) #сохранение повернутого изображения из матрицы

    x = int( center[0] - width/2  ) #поиск координаты x слева
    y = int( center[1] - height/2 ) # поиск координаты у снизу

    image = image[ y:int(y+height), x:int(x+width) ] #обрезка изображения по координатам

    cv2.imwrite(f'{path}/{name_file}', image) #path -- путь к корневой дирректории
    
    #возвращает обрезанное изображение
    return image

In [ ]:
image_1 = subimage("pic_1",(0.5,0.6),3.5,0.6,0.2)
image_2 = subimage("pic_2",(0.5,0.6),3.5,0.6,0.2)
image_3 = subimage("pic_3",(0.5,0.6),3.5,0.6,0.2)
image_4 = subimage("pic_4",(0.5,0.6),3.5,0.6,0.2)

# Formula for bringing $\textbf{data to a convenient view}$. PREOB() function

# Table $\to$ row by mean value of coluns

$x_i = log_{10}\dfrac{65536}{x_i}\times1000$, if we have pixels value range (0,65536)

$x_i = log_{10}\dfrac{257.003921569}{x_i}\times1000$, if we have pixels value range (0,255)

$x_i$ -- pixel value, 65536 -- maximum possible value. 

We don't want to work with values like 26000 or smth like that, so we divide max pos value on our output value and take $log_{10}$. After that we get values of the order of 3-4. As soon as it too smal values and for axis stretching we multiply our value on 1000.

### Function

In [ ]:
def preob(image):
    """
    function recalculate values and make it more comfortable to vizualization and calculate mean value in each column, 
    after that it write in one-dementional array 
    """
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#переводим изображение в черно-белый формат, так как цветное изображение является трехмерным массивом
    im_array = np.array(image)# создаем из нее массив 
    array_log10 = np.log10(257.003921569/im_array) * 1000 #(257.003921569 = 65536/255), also data (смещение запятой)
    mean = array_log10.mean(axis=0) #среднее значение по столбцам
    #возвращает массив из средних значений
    return mean

### Implementation

In [ ]:
mean_call1 = preob(image_1)
mean_call2 = preob(image_2)
mean_call3 = preob(image_3)
mean_call4 = preob(image_4)

### Save txt with mean values

In [ ]:
np.savetxt('Files/call1_mean.txt', mean_call1)
np.savetxt('Files/call2_mean.txt', mean_call2)
np.savetxt('Files/call3_mean.txt', mean_call3)
np.savetxt('Files/call4_mean.txt', mean_call4)

# Graph

In [ ]:
fig1 = plt.figure(figsize=(8,4),dpi = 100)#(figsize=(14,7),dpi = 200)
x = np.arange(1, len(mean_call1) + 1)
plt.scatter(x,mean_call1,color = 'orangered', s = 2, alpha = 0.5)
plt.title('call1')
fig2 = plt.figure(figsize=(8,4),dpi = 100)#(figsize=(14,7),dpi = 200)
x = np.arange(1, len(mean_call2) + 1)
plt.scatter(x,mean_call2,color = 'orangered', s = 2, alpha = 0.5)
plt.title('call2')
fig3 = plt.figure(figsize=(8,4),dpi = 100)#(figsize=(14,7),dpi = 200)
x = np.arange(1, len(mean_call3) + 1)
plt.scatter(x,mean_call3,color = 'orangered', s = 2, alpha = 0.5)
plt.title('call3')
fig4 = plt.figure(figsize=(8,4),dpi = 100)#(figsize=(14,7),dpi = 200)
x = np.arange(1, len(mean_call4) + 1)
plt.scatter(x,mean_call4,color = 'orangered', s = 2, alpha = 0.5)
plt.title('call4')

# Peaks finding

In [ ]:
def peaks(mean,order):
    """
    function for searching peaks in array 
    """
    peak_indexes = signal.argrelextrema(mean, np.greater, order=order)[0] #индексы пиков 
    peak_mean =[] # создаем пустой массив для средних значений пиков
    for i in range(0,len(mean)):
        if i in peak_indexes:
            peak_mean.append((mean[i-2]+mean[i-1]+mean[i]+mean[i+1]+mean[i+2])/5) # ищем среднее по пяти рядом стоящих значений
    #возвращает позицию/индекс найденного пика и среднее значение по пяти значениям : пик + 2 слева + 2 справа
    return peak_indexes,peak_mean

In [ ]:
peak_indexes_1,peak_mean_1 = peaks(mean_call1,order=5)
peak_indexes_2,peak_mean_2 = peaks(mean_call2,order=5)
peak_indexes_3,peak_mean_3 = peaks(mean_call3,order=5)
peak_indexes_4,peak_mean_4 = peaks(mean_call4,order=5)